In [1]:
import base64
import os
from datetime import datetime, timedelta
from urllib.parse import urlencode, urlparse, parse_qs
from abc import ABC, abstractmethod, abstractproperty
import webbrowser
import requests

    
class SpotifyClient:
    BASE_URL = 'https://api.spotify.com/v1'
    
    def __init__(self, client_id, client_secret, redirect_uri=None, scope=None, state=None):
        self._auth_manager = AuthManager(client_id, client_secret, redirect_uri, scope)
        
    @property
    def client_credentials(self):
        client_credentials = f"{self._client_id}:{self._client_secret}"
        self._client_credentials = base64.b64encode(client_credentials.encode())
        return self._client_credentials.decode()
    
    @property
    def token_info(self):
        return self._token_info
    
    @token_info.setter
    def token_info(self, val):
        # Assert if this is a valid token?
        self._token_info = val

    @property
    def auth_manager(self):
        return self._auth_manager
    
    @auth_manager.setter
    def auth_manager(self, val):
        if isinstance(val, AuthManager):
            self._auth_manager = val

    def authorise(self):
        self.token_info = self.auth_manager.request_token()
    
    @property
    def resource_headers(self):
        access_token = self.token_info.get('access_token')
        return {'Authorization':f'Bearer {access_token}'}
    
    # TODO  
    def get_resource(self, lookup_id, resource_type='albums'):
        endpoint = f'{self.BASE_URL}/{resource_type}/{lookup_id}'
        headers = self.resource_headers
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, 'albums')
    
    
    def get_artist(self, _id):
        return self.get_resource(_id, 'artists')
    
    
    def search(self, query, search_type='track'):
        endpoint = f'{self.BASE_URL}/search'
        headers = self.resource_headers
        data = urlencode({'q':query,'type':search_type.lower()})
        url = f'{endpoint}?{data}'
        print(url)
        r = requests.get(url, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
        
    
class AuthManager:
    TOKEN_URL = "https://accounts.spotify.com/api/token"
    AUTH_URL = "https://accounts.spotify.com/authorize"
    
    def __init__(self, client_id, client_secret, redirect_uri=None, scope=None, state=None):
        self._client_id = client_id
        self._client_secret = client_secret
        self._redirect_uri = redirect_uri
        self._scope = scope
        self._state = state
    
    @property
    def client_id(self):
        return self._client_id
    
    @property
    def client_secret(self):
        return self._client_secret
    
    @property
    def redirect_uri(self):
        return self._redirect_uri
    
    @property
    def scope(self):
        return self._scope
    
    @property
    def state(self):
        return self._state
    
    @property
    def client_credentials(self):
        client_credentials = f"{self._client_id}:{self._client_secret}"
        self._client_credentials = base64.b64encode(client_credentials.encode())
        return self._client_credentials.decode()
    
    @property
    def redirect_uri(self):
        return self._redirect_uri
    
    # Add setter for scope and validate
    @property
    def scope(self):
        return self._scope
    
    @property
    def grant_type(self):
        if not self._redirect_uri:
            return {"grant_type": "client_credentials"}
        return {"grant_type": "authorization_code"}

    @property
    def basic_auth(self):
        return {"Authorization": f"Basic {self.client_credentials}"}
    
    @property
    def content_type(self):
        return {'content_type':'application/x-www-form-urlencoded'}
    
    @property
    def token_data(self):
        return {'code':self.code, 'state':self.state, 'redirect_uri':self.redirect_uri} | self.grant_type
        
    # Change this to just be auth when merging the basic and bearer auth tokens
    @property
    def token_headers(self):
        return self.basic_auth | self.content_type
    
    @property
    def auth_url(self):
        query = {
            "client_id": self.client_id,
            "response_type": 'code',
            "redirect_uri": self.redirect_uri,
        }
        if self.scope:
            query["scope"] = self.scope
        if self.state is None:
            query["state"] = self.state
        # OPTIONAL: if instance has show dialog option, add this to the query
        urlparams = urlencode(query)
        return f"{self.AUTH_URL}?{urlparams}"
    
    def authorise(self):
        url = webbrowser.open(self.auth_url)
        print(url)
        # _, code = AuthManager.parse_auth_url(url)
    
    def request_token(self):
        url = self.TOKEN_URL
        data = self.token_data
        headers = self.token_headers
        r = requests.post(url, data=data, headers=headers)
        if r.status_code not in range(200, 299):
            raise Exception('Could not authenticate user, please try again')
        return r.json()
    
    @staticmethod
    def parse_auth_url(url):
        query = urlparse(url).query
        form = parse_qs(query)
        if "error" in form:
            raise Exception("Error authorising user")
        return tuple(form.get(param) for param in ["state", "code"])
        
    @property
    def code(self):
        return self._code
    
    @code.setter
    def code(self, val):
        self._code = val
        
    @property
    def state(self):
        return self._state

In [2]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
redirect_uri = os.getenv('REDIRECT_URI')

c = SpotifyClient(client_id, client_secret, redirect_uri)

In [3]:
webbrowser.open(c.auth_manager.auth_url)

True

In [7]:
c.auth_manager.redirect_uri

'http://127.0.0.1:8501'

In [47]:
c.auth_manager.request_token()

AttributeError: 'AuthManager' object has no attribute '_code'

In [5]:
# TODO
# - Figure out authorisation flow, when there is scope and redirect uri and add to 
#     AuthManager (will need to test this in Flask or copy/paste the URL to parse response query)
# - Update abstract AuthManager class, so there is a client credentials AuthManager and an authorisation flow manager
# - Attempt fetching user data with access token created through oauth